In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0+cu118
tiktoken version: 0.9.0


### 2.1 Embedding

### 2.2 Tokenizing text

- 最小化的 表意 单元
- 最小 化的 表意单 元

In [3]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
	url = ("https://raw.githubusercontent.com/rasbt/"
		"LLMs-from-scratch/main/ch02/01_main-chapter-code/"
		"the-verdict.txt")
	file_path = "the-verdict.txt"
	urllib.request.urlretrieve(url, file_path)

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
	raw_text = f.read()
print(f"总字符数: {len(raw_text)}")
print(raw_text[:99])

总字符数: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
# use whitespace as the delimiter

import re

text = "Hello, world. This, is a test."
result = re.split(r"(\s)", text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [6]:
# use , and . and whitespace as delimiters

result = re.split(r"([,.]|\s)", text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [7]:
# remove empty strings
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [8]:
# more powerful 
# split with multiple delimiters

text = "Hello, world. Is this-- a test?"

result = re.split(r"([,.:;?_!\"()']|--|\s)", text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [9]:
# 将以上分词实际应用于文本
preprocessed = re.split(r"([,.:;?!_\"()']|--|\s)", raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
# 前面的.strip() 可以去掉前导的以及尾随的空格
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


### 2.3 token ID

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
# mapping
vocab = {token:integer for integer, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break


('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [12]:
class SimleTokenizerV1:
	def __init__(self, vocab) -> None:
		# 用vocab 建立一个映射表
		self.str_to_int = vocab
		self.int_to_str = {int:stri for stri, int in vocab.items()}

	def encode(self, text):
		# 将text进行分词并映射为整数ID
		preprocessed = re.split(r"([,.:;?!_()\"']|--|\s)", text)
		preprocessed = [item.strip() for item in preprocessed if item.strip()]
		ids = [self.str_to_int[s] for s in preprocessed]
		return ids
	
	def decode(self, ids):
		text = " ".join([self.int_to_str[i] for i in ids])
		# 去掉多余的空格
		text = re.sub(r"\s+([,.?!\"()'])", r"\1", text)
		return text

In [13]:
tokenizer = SimleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
			Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [15]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

### 2.4 adding special tokens

- 特殊的token标记

	- [BOS] beginning of sentence 开始

	- [EOS] end of sentence 结束

	- [PAD] padding 填充

	- [UNK] unknown 表中未有

- <|endoftext|> 结束符 分隔两个文本

In [16]:
tokenizer = SimleTokenizerV1(vocab)
text = "Hello, do you like tea. Is this-- a test?"
tokenizer.encode(text)
# Hello 未在表中


KeyError: 'Hello'

In [17]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}
len(vocab)

1132

In [18]:
for i, item in enumerate(list(vocab.items())[-5:]):
	print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [19]:
class SimleTokenizerV2(SimleTokenizerV1):
	def __init__(self, vocab) -> None:
		super().__init__(vocab)

	def encode(self, text):
		preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
		preprocessed = [item.strip() for item in preprocessed if item.strip()]
		preprocessed = [
			item if item in self.str_to_int else "<|unk|>" 
				for item in preprocessed
		]

		ids = [self.str_to_int[s] for s in preprocessed]
		return ids
	
	def decode(self, ids):
		return super().decode(ids)

In [20]:
tokenizer = SimleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join([text1, text2])

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [21]:
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


### 2.5 BytePair encoding

- BPE 会把未知<unk>划分为已知的单词或字母

In [7]:
import importlib.metadata
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [25]:
tokenizer = tiktoken.get_encoding("gpt2")

In [26]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [27]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


### 2.6 data sampling with a sliding window

逐词预测

In [28]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
	raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

enc_sample = enc_text[50:]

5145


In [29]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [30]:
for i in range(1, context_size+1):
	context = enc_sample[:i]
	desired = enc_sample[i]
	print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


- 数据加载器

In [6]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu118


In [1]:
from torch.utils.data import Dataset

class GPTDatasetV1(Dataset):
	def __init__(self, txt, tokenizer, max_length, stride) -> None:
		self.input_ids = []
		self.target_ids = []
		
		token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
		assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

		for i in range(0, len(token_ids) - max_length, stride):
			input_chunk = token_ids[i:i + max_length]
			target_chunk = token_ids[i+1: i+max_length + 1]
			self.input_ids.append(torch.tensor(input_chunk))
			self.target_ids.append(torch.tensor(target_chunk))

	def __len__(self):
		return len(self.input_ids)
	
	def __getitem__(self, idx):
		return self.input_ids[idx], self.target_ids[idx]

In [3]:
from torch.utils.data import DataLoader

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128,
						shuffle=True, drop_last=True, num_workers=0):
	tokenizer = tiktoken.get_encoding("gpt2")
	dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

	dataloader = DataLoader(
		dataset,
		batch_size=batch_size,
		shuffle=shuffle,
		drop_last=drop_last,
		num_workers=num_workers
	)
	
	return dataloader

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
	raw_text = f.read()

In [ ]:
dataloader = create_dataloader_v1(
	raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)	# input, target

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
# 更大的stride

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### 2.7 Creating token embeddings

In [ ]:
# torch.nn.embedding == torch.nn.functional.one_hot + torch.nn.linear

In [ ]:
from torch import embedding

# 输入
input_ids = torch.tensor([2, 3, 5, 1])

# 编码器
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [14]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


### 2.8 Encoding word positions

位置编码

In [18]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_length = 4
# stride 4 不重复
dataloader = create_dataloader_v1(
	raw_text, batch_size=8, max_length=max_length,
	stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [21]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
# absolute position
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
print(pos_embedding_layer.weight)
print(pos_embedding_layer.weight.shape)

Parameter containing:
tensor([[-0.9353,  0.3188, -0.7261,  ..., -0.0690, -1.2632, -0.0921],
        [-2.5158, -0.5055, -0.8398,  ..., -2.8479,  0.4933,  0.3542],
        [ 0.5502, -0.4216,  0.9218,  ..., -0.0566, -0.6754,  0.0693],
        [-0.4435, -0.4947, -1.0855,  ..., -1.1526, -1.0944, -0.6755]],
       requires_grad=True)
torch.Size([4, 256])


In [27]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

embedding_result = token_embeddings + pos_embeddings

torch.Size([4, 256])
